<a href="https://colab.research.google.com/github/Dec-35/mathsDecryptage/blob/main/TP_1_Cesar_et_affine.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# TP 1 - Chiffrement symétrique (1)

La <b>cryptographie</b> est un ensemble de techniques permettant de transformer un <i>message clair</i> en <i>message chiffré</i> qui ne peut pas être déchiffré sans la connaissance d'une clé secrète.

Nous allons commencer l'étude de différents algortihmes de chiffrement par les algorithmes de chiffrement <b>symétrique</b>. Le principe général de ces algorithmes est basé sur le partage d'une unique clé secrète entre l'expéditeur et le destinataire. Cette clé va servir à chiffrer un message mais aussi à le déchiffrer, il est donc indispensable qu'elle ne soit pas divulguée.

### IMPORTANT :

Avant de commencer le TP, nous allons charger les différentes fonctions que vous avez écrites lors du TP1 - Arithmétique pour la cryptographie, et qui nous serons utile. Pour celà, il vous suffit de créer un fichier intitulé <code>OutilsCrypto.py</code>, d'y coller le code des différentes fonctions du TP1, de le sauvegarder au même endroit que ce TP, et de le charger en exécutant le bloc suivant.

In [1]:
from google.colab import drive
drive.mount('/content/drive')
import sys
sys.path.append('/content/drive/MyDrive/Colab Notebooks/BUT 2/R3.09 - Crypto')

ModuleNotFoundError: No module named 'google'

In [2]:
from OutilsCrypto import *

À la fin de chaque TP, vous pourrez compléter le fichier <code>OutilsCrypto.py</code> en y ajoutant les nouvelles fonctions écrites.

## 1. Préparation du texte à chiffrer
Pour chiffrer un message, quel que soit l'algorithme choisi, il faut souvent commencer par le transformer pour pouvoir lui appliquer des opérations mathématiques.<br>

### 1.1 Filtre : enlever les caractères spéciaux

La première étape consiste en général à <i>nettoyer</i> le texte en se débarrassant des accents, des signes de ponctuation, des caractères spéciaux, etc.

<b> Exercice 1.</b> Écrire la fonction <code>Filtre(txt)</code> permettant de gérer cet aspect. Elle doit transformer n'importe quelle chaîne de caractères en une chaîne en minuscule ne contenant que les lettres de l'alphabet non accentuées.<br>
Par exemple, "Bonjour, ça va ?" doit devenir : "bonjourcava"<br>
<i>Indication :</i> Vous pourrez utiliser la fonction <code>.lower()</code> ainsi que la fonction <code>ord()</code>.

In [3]:
def Filtre(txt) :
    res=""
    for c in txt.lower() :
        if(ord('a')<=ord(c)<=ord('z')) : res+=c
        if(c=='à' or c=='â' or c=='ä' or c=='á' or c=='å') : res+='a'
        elif(c=='ç') : res+='c'
        elif(c=='é' or c=='è' or c=='ê' or c=='ë') : res+='e'
        elif(c=='ï' or c=='î' or c=='ì' or c=='í') : res+='i'
        elif(c=='ö' or c=='ò' or c=='ô' or c=='ø' or c=='ó') : res+='o'
        elif(c=='û' or c=='ü' or c=='ù' or c=='ú') : res+='u'
        elif(c=='æ') : res+='ae'
        elif(c=='œ') : res+='oe'
        elif(c=='ÿ') : res+='y'
        elif(c=='ñ') : res+='n'
        elif(c=='ß') : res+='ss'
    return res

Tester votre fonction à l'aide du bloc suivant.

In [4]:
txt = input("texte = ")
Filtre(txt)

'rien'

### 1.2 Codage et décodage d'un caractère en nombre

L'étape suivante consiste à écrire une fonction <code>code(c)</code> qui associe un nombre à chaque lettre <code>c</code> afin de pouvoir appliquer des opérations mathématiques. Nous utiliserons ici le procédé le plus simple qui consiste à associer <b>0</b> à <b>a</b>, <b>1</b> à <b>b</b>, ... , <b>25</b> à <b>z</b>.<br>
Nous utliserons également la fonction inverse <code>decode(n)</code> qui consistera à retrouver une lettre à partir du nombre qui lui est associé.

<b> Exercice 2.</b> Écrire la fonction <code>code(c)</code> qui prend en paramètre un caractère <code>c</code> et qui renvoie le nombre associé. Elle doit envoyer $-1$ si $c$ n'est pas un caractère autorisé (i.e. une lettre non accentuée entre a et z)

In [6]:
#Code erreur : -1
def code(c) :
    res = ""
    res = Filtre(c)
    if res == "" :
      return -1

    return ord(res) -  97 #décalage unicode

<b> Exercice 3.</b> Écrire la fonction <code>decode(n)</code> qui prend en paramètre un entier <code>n</code> et qui renvoie le caractère associé. Elle doit envoyer "?" si $n$ n'est pas un nombre autorisé (i.e. un nombre $0\leq n\leq 25$)

In [7]:
#Code erreur : '?'
def decode(n) :
    # à compléter
    if(0 <= n <= 25) :
      n = n + 97
      return chr(n)
    return '?'

Tester ces fonctions à l'aide du bloc de code suivant.

In [8]:
print(code("a")) # on attend 0
print(code("h")) # on attend 7
print(code("!")) # on attend -1
print(decode(0)) # on attend a
print(decode(5)) # on attend f
print(decode(-1)) # on attend ?

0
7
-1
a
f
?


## 2. Chiffrement de César

Le chiffrement de César, appelé aussi chiffrement à décalage, consiste à décaler chaque lettre du message de $k$ lettres. Ainsi, si on choisit $k=3$ par exemple, <b>a</b> devient <b>d</b>, <b>b</b> devient <b>e</b>, etc. Quand on arrive à la fin de l'alphabet, on repart du début : <b>x</b> devient <b>a</b>, <b>y</b> devient <b>b</b> et <b>z</b> devient <b>c</b>.<br>
Cette opération de décalage s'exprime très bien sur les nombres associés aux lettres par le procédé de codage détaillé ci-dessus.<br>
$$
\begin{array}{rccc}
f_k\ :\ & \mathbb{Z}/26\mathbb{Z} & \longrightarrow & \mathbb{Z}/26\mathbb{Z}\\
& x & \longmapsto & x+k
\end{array}
$$
En effet, si on part de la lettre <b>z</b>, elle est codée par 25, on calcule $$f_3(25) \equiv 25+3 \equiv 28 \equiv 2 \pmod{26}$$ et 2 est décodé en <b>c</b>.

<b> Exercice 4.</b> Sur papier, chiffrer le mot "bonjour" en utilisant le chiffrement de César avec la clé 3. Indiquer votre réponse en éditant puis exécutant le bloc suivant.

<div style="background-color:rgba(0, 255, 0, 0.19);padding:3%;">
<b>Réponse : </b> Le texte "bonjour" chiffré avec le chiffrement de César de clé 3 est <b>erqmrxu</b>
</div>

Écrire la fonction <code>chiffreCesar(mot,cle)</code> qui a deux paramètres obligatoires :
<ul>
    <li><code>mot</code> : une chaîne de caractères</li>
    <li><code>cle</code> : un entier</li>
</ul>
Cette fonction renvoie une chaîne de caractères correspondant au chiffrement de César de <code>mot</code> avec la clé <code>cle</code>.<br>

La fonction renvoie la chaine vide (ie $\texttt{""}$) en cas d'erreur.
</p>

In [9]:
def chiffreCesar(mot,k):
    res = ""
    for c in mot :
        res += decode((code(c)+k)%26)
    return res

Tester votre fonction à l'aide du bloc de code suivant.

In [10]:
print("*** CHIFFREMENT ***")
txt = "bonjour"
cle = 3
texte_chiffre = chiffreCesar(txt,cle)
print("Le texte '",txt,"' chiffré avec le chiffrement de César de clé",cle," est : ",texte_chiffre)

*** CHIFFREMENT ***
Le texte ' bonjour ' chiffré avec le chiffrement de César de clé 3  est :  erqmrxu


Sur papier, déchiffrer le mot "hbylcvpy" en sachant qu'il a été chiffré à l'aide du chiffrement de César avec la clé 7. Indiquer votre réponse en éditant puis exécutant le bloc suivant.

<div style="background-color:rgba(0, 255, 0, 0.19);padding:3%;">
<b>Réponse : </b> Le texte "hbylcvpy" déchiffré avec le chiffrement de César de clé 7 est <b>aurevoir</b>
</div>

Écrire la fonction <code>dechiffreCesar(mot,cle)</code> qui permet de déchiffrer un <code>mot</code> qui a été chiffré avec la clé <code>clé</code>.

In [11]:
def dechiffreCesar(mot,k):
    res = ""
    for c in mot :
        res += decode((code(c)-k)%26)
    return res

Tester votre fonction à l'aide du bloc de code suivant.

In [12]:
print("*** DECHIFFREMENT ***")
txt = "hbylcvpy"
cle = 7
texte_clair = dechiffreCesar(txt,cle)
print("Le texte '",txt,"' déchiffré avec le chiffrement de César de clé",cle," est : ",texte_clair)

*** DECHIFFREMENT ***
Le texte ' hbylcvpy ' déchiffré avec le chiffrement de César de clé 7  est :  aurevoir


<b>Un peu de cryptanalyse. </b> Écrire la fonction <code>decrypteCesar(mot)</code> qui affiche toutes les possibilités de déchiffrement de <code>mot</code> par brute force.

In [23]:
def decrypteCesar(mot):
    for i in range(26) :
        for c in mot :
            print(decode((code(c)-i)%26), end="")
        print(' avec la clé', i)

Dans le bloc suivant, utilisez la fonction <code>decrypteCesar(mot)</code> pour décrypter le texte <b>prfgdhnaqzrzrcnfgreevoyryrpuvsserzragqrprfne</b>

In [24]:
decrypteCesar("prfgdhnaqzrzrcnfgreevoyryrpuvsserzragqrprfne")

prfgdhnaqzrzrcnfgreevoyryrpuvsserzragqrprfne avec la clé 0
oqefcgmzpyqyqbmefqddunxqxqoturrdqyqzfpqoqemd avec la clé 1
npdebflyoxpxpaldepcctmwpwpnstqqcpxpyeopnpdlc avec la clé 2
mocdaekxnwowozkcdobbslvovomrsppbowoxdnomockb avec la clé 3
lnbczdjwmvnvnyjbcnaarkununlqrooanvnwcmnlnbja avec la clé 4
kmabycivlumumxiabmzzqjtmtmkpqnnzmumvblmkmaiz avec la clé 5
jlzaxbhuktltlwhzalyypislsljopmmyltluakljlzhy avec la clé 6
ikyzwagtjskskvgyzkxxohrkrkinollxksktzjkikygx avec la clé 7
hjxyvzfsirjrjufxyjwwngqjqjhmnkkwjrjsyijhjxfw avec la clé 8
giwxuyerhqiqitewxivvmfpipiglmjjviqirxhigiwev avec la clé 9
fhvwtxdqgphphsdvwhuuleohohfkliiuhphqwghfhvdu avec la clé 10
eguvswcpfogogrcuvgttkdngngejkhhtgogpvfgeguct avec la clé 11
dfturvboenfnfqbtufssjcmfmfdijggsfnfouefdftbs avec la clé 12
cestquandmemepasterriblelechiffrementdecesar avec la clé 13
bdrsptzmcldldozrsdqqhakdkdbgheeqdldmscdbdrzq avec la clé 14
acqrosylbkckcnyqrcppgzjcjcafgddpckclrbcacqyp avec la clé 15
zbpqnrxkajbjbmxpqboofyibibzefccobjbkqabzbpxo avec 

<b> Question : </b>Combien de résultats différents obtient-on ? Retrouver la clé et le texte décrypté.

<div style="background-color:rgba(0, 255, 0, 0.19);padding:3%;">
<b>Réponse : </b> Il y a <b>26</b> résultats à analyser. La clé est <b>13</b> et le texte décrypté est <b>cestquandmemepasterriblelechiffrementdecesar</b>
</div>

## 3. Chiffrement affine

Le chiffrement affine est une modification de l'algorithme de César qui consiste à choisr comme clé un couple d'entiers $(a,b)$ et à appliquer à chaque nombre $x$ codant un caractère $c$ la transformation affine $ax+b \pmod{26}$.<br>
$$
\begin{array}{rccc}
f_{a,b}\ :\ & \mathbb{Z}/26\mathbb{Z} & \longrightarrow & \mathbb{Z}/26\mathbb{Z}\\
& x & \longmapsto & ax+b
\end{array}
$$
Par exemple, avec la clé $(3,7)$ si on part de la lettre <b>z</b>, elle est codée par 25, on calcule $$f_{3,7}(25) \equiv 3*25+7 \equiv 82 \equiv 4 \pmod{26}$$ et 4 est décodé en <b>e</b>.

<b> Exercice 5.</b> Sur papier, chiffrer le mot "bonjour" en utilisant le chiffrement affine de clé $(3,7)$. Indiquer votre réponse en éditant puis exécutant le bloc suivant.

<div style="background-color:rgba(0, 255, 0, 0.19);padding:3%;">
<b>Réponse : </b> Le texte "bonjour" chiffré avec le chiffrement affine de clé $(3,7)$ est <b>kxuixpg</b>
</div>

Écrire la fonction <code>chiffreAffine(mot,a,b)</code> qui a deux paramètres obligatoires :
<ul>
    <li><code>mot</code> : une chaîne de caractères</li>
    <li><code>a,b</code> : 2 entiers</li>
</ul>
Cette fonction renvoie une chaîne de caractères correspondant au chiffrement affine de <code>mot</code> avec la clé <code>(a,b)</code>.<br>

La fonction renvoie la chaine vide (ie $\texttt{""}$) en cas d'erreur.
</p>

In [30]:
def chiffreAffine(mot,a,b):
    # a * mot + b

    res = ""
    for c in mot :
        res += decode((a * code(c) + b)%26)
    return res

Tester votre fonction à l'aide du bloc suivant.

In [32]:
print("*** CHIFFREMENT ***")
txt = "bonjour"
cle_a = 3
cle_b = 7
texte_chiffre = chiffreAffine(txt,cle_a,cle_b)
print("Le texte '",txt,"' chiffré avec le chiffrement affine de clé (",cle_a,",",cle_b,") est : ",texte_chiffre)

*** CHIFFREMENT ***
Le texte ' bonjour ' chiffré avec le chiffrement affine de clé ( 3 , 7 ) est :  kxuixpg


Pour déchiffrer, nous avons besoin d'inverser la fonction $f_{a,b}$, c'est-à-dire de pouvoir trouver pour chaque $y\in\mathbb{Z}/26\mathbb{Z}$ un unique $x\in\mathbb{Z}/26\mathbb{Z}$ tel que $ax+b \equiv y \pmod{26}$.<br>
Malheureusement, ceci n'est pas toujours possible !<br>
<b>Question : </b>Trouver à quelle condition sur $a$ il est possible d'écrire une fonction de déchiffrement et indiquer alors quelle fonction permet de calculer $x$ en fonction de $y$.

<div style="background-color:rgba(0, 255, 0, 0.19);padding:3%;">
<b>Réponse : </b> Il est possible d'écrire une fonction de déchiffrement si et seulement si <b>a</b> est premier avec 26. Dans ce cas, on peut calculer <b>x</b> en fonction de <b>y</b> en utilisant la fonction <code>invMod(a,26)</code> du TP0.
</div>

Sur papier, déchiffrer le mot "wvspayinlc", sachant qu'il a été chiffré en utilisant le chiffrement affine de clé $(5,8)$. Indiquer votre réponse en éditant puis exécutant le bloc suivant.

<div style="background-color:rgba(0, 255, 0, 0.19);padding:3%;">
<b>Réponse : </b> Le texte "wvspayinlc" déchiffré avec le chiffrement affine de clé (5,8) est <b>incroyable</b>
</div>

Écrire la fonction <code>dechiffreAffine(mot,a,b)</code> qui permet de déchiffrer un mot qui a été chiffré avec la clé <code>(a,b)</code>. Si ce n'est pas possible, afficher un message expliquant que la clé n'est pas une clé de chiffrement affine autorisée et retourner une chaîne vide.

In [37]:
def dechiffreAffine(mot,a,b):
    # cas impossible
    if invMod(a,26) == -1 :
        return "Impossible"

    res = ""
    for c in mot :
        res += decode((invMod(a,26) * (code(c) - b))%26)
    return res

Tester votre fonction à l'aide du bloc de code suivant.

In [38]:
print("*** DECHIFFREMENT ***")
txt = "wvspayinlc"
cle_a = 5
cle_b = 8
texte_clair = dechiffreAffine(txt,cle_a,cle_b)
print("Le texte '",txt,"' chiffré avec le chiffrement affine de clé (",cle_a,",",cle_b,") est : ",texte_clair)

*** DECHIFFREMENT ***
Le texte ' wvspayinlc ' chiffré avec le chiffrement affine de clé ( 5 , 8 ) est :  incroyable


<b>Un peu de cryptanalyse. </b> Écrire la fonction <code>decrypteAffine(mot)</code> qui affiche toutes les possibilités de chiffrement de <code>mot</code> par brute force.

In [49]:
def decrypteAffine(mot):
    nbres = 0
    for a in range(26):
        if invMod(a,26) != -1 :
            for b in range(26):
                nbres += 1
                res = ""
                for c in mot :
                    res += decode((invMod(a,26) * (code(c) - b))%26)
                print(res, ' avec le couple ', a,b)
    print(nbres , ' résultats')
        

Dans le bloc suivant, utilisez la fonction <code>decrypteCesar(mot)</code> pour décrypter le texte <b>YZDGRKKMZJZUIHKKRUZZXIJRZTAJHRXRYZXIZUDFMZKHDRYZHHIIHBTZMZUSMTIZKFMDZ</b>

In [50]:
decrypteAffine("YZDGRKKMZJZUIHKKRUZZXIJRZTAJHRXRYZXIZUDFMZKHDRYZHHIIHBTZMZUSMTIZKFMDZ")

0  n'est pas inversible modulo  26
yzdgrkkmzjzuihkkruzzxijrztajhrxryzxizudfmzkhdryzhhiihbtzmzusmtizkfmdz  avec le couple  1 0
xycfqjjlyiythgjjqtyywhiqyszigqwqxywhytcelyjgcqxygghhgasylytrlshyjelcy  avec le couple  1 1
wxbepiikxhxsgfiipsxxvghpxryhfpvpwxvgxsbdkxifbpwxffggfzrxkxsqkrgxidkbx  avec le couple  1 2
vwadohhjwgwrfehhorwwufgowqxgeouovwufwracjwheaovweeffeyqwjwrpjqfwhcjaw  avec le couple  1 3
uvzcnggivfvqedggnqvvtefnvpwfdntnuvtevqzbivgdznuvddeedxpvivqoipevgbizv  avec le couple  1 4
tuybmffhueupdcffmpuusdemuovecmsmtusdupyahufcymtuccddcwouhupnhodufahyu  avec le couple  1 5
stxaleegtdtocbeelottrcdltnudblrlstrctoxzgtebxlstbbccbvntgtomgnctezgxt  avec le couple  1 6
rswzkddfscsnbaddknssqbcksmtcakqkrsqbsnwyfsdawkrsaabbaumsfsnlfmbsdyfws  avec le couple  1 7
qrvyjccerbrmazccjmrrpabjrlsbzjpjqrparmvxerczvjqrzzaaztlrermkelarcxevr  avec le couple  1 8
pquxibbdqaqlzybbilqqozaiqkrayioipqozqluwdqbyuipqyyzzyskqdqljdkzqbwduq  avec le couple  1 9
optwhaacpzpkyxaahkppnyzhpjqzxhnhopnypktvcpaxthopxxyyxrj

<b> Question : </b>Combien de résultats différents obtient-on ? Retrouver la clé et le texte décrypté.

<div style="background-color:rgba(0, 255, 0, 0.19);padding:3%;">
<b>Réponse : </b> Il y a 312 résultats à analyser. La clé est 11,7 et le texte décrypté est <b>lechiffrementaffineestmieuxmaisilestencorefacileaattaquerenbruteforce</b>
</div>